In [15]:
# Import Module 
from win32com import client 
  
# Open Microsoft Excel 
excel = client.Dispatch("Excel.Application") 
  
# Read Excel File 
sheets = excel.Workbooks.Open('Excel File Path') 
work_sheets = sheets.Worksheets[0] 
  
# Convert into PDF File 
work_sheets.ExportAsFixedFormat(0, 'PDF File Path') 

In [1]:
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [3]:
import os

In [5]:
os.environ['DB_PASSWORD']

'h1R1m!55'

In [6]:
data = {
    "abc" : "123",
    "def" : "456"
}

In [8]:
data['fed']

KeyError: 'fed'

In [10]:
data

{'abc': '123', 'def': '456'}

In [9]:
data.get('fed')